# Datenbankzugriff

## Sqlite

Mit dem Python-Paket [sqlite3](https://docs.python.org/library/sqlite3.html)
kann auf  sqlite-Datenbanken direkt zugegriffen werden. Es muss daher nicht
installiert werden und ist bei jeder Python-Installation vorhanden.

In der folgenden Beispielanwendung erstellen wir eine Beispieltabelle,
fügen ein paar Daten hinzu und geben diese wieder aus.

In [1]:
import sqlite3

Nach dem Import des `sqlite3`-Moduls können nun Daten hinzugefügt und anschließend wieder ausgelesen werden.
Zunächst wird eine Verbindung erstellt und aus dieser eine `Cursor` generiert, mit dem auf die Datenbank
zugegriffen werden kann.

In [2]:
def insert_into_db():
    conn = sqlite3.connect("datenbank.db")
    c = conn.cursor()
    c.execute("CREATE TABLE IF NOT EXISTS personen(nr int, name text)")

    for i, name in enumerate(["Peter", "Petra", "Hans", "Claudia"]):
        # use ? to avoid sql injection
        c.execute("INSERT INTO personen VALUES(?,?)", (100+i, name))

    conn.commit()
    conn.close()

insert_into_db()

Wir finden nun eine Datei `datenbank.db` im aktuellen Verzeichnis.

In [13]:
%ls datenbank.db

datenbank.db


Nun können die Daten wieder aus der Datei gelesen werden.

In [4]:
def select_from_db():
    conn = sqlite3.connect("datenbank.db")
    c = conn.cursor()
    rows = c.execute("SELECT nr, name FROM personen")

    print("Nr.\tName")
    for i, name in rows:
        print(i, "    ", name)

    conn.close()

select_from_db()


Nr.	Name
100      Peter
101      Petra
102      Hans
103      Claudia


Zum Schluss werden alle Daten wieder gelöscht.

In [5]:
def del_data():
    conn = sqlite3.connect("datenbank.db")
    c = conn.cursor()
    c.execute("DELETE FROM personen")
    
    conn.commit()
    conn.close()
    
del_data()

## MySQL

Für den Zugriff auf eine MySQL-Datenbank muss zusätzlich ein 
[Connector für 
MySQL](https://dev.mysql.com/doc/connector-python/en/connector-python-installation-binary.html)
installiert werden. Dies kann z.B. mit dem folgenden Aufruf geschehen.

    $ pip install mysql-connector-python

Die weitere Dokumentation von MySQL enthält verschiedene [Beispiele, die
den Umgang mit einer MySQL-Datenbank 
zeigen](https://dev.mysql.com/doc/connector-python/en/connector-python-examples.html).
Der wesentliche Unterschied besteht darin, wie eine Verbindung aufgebaut wird.
Nach dem Erzeugen eines Cursors mit ``cursor = eine_connection.cursor()`` kann
mit einem Aufruf von ``cursor.execute(...)`` wie im Falle von sqlite auf die 
Datenbank zugegriffen werden.

Das sehr ausführliche [Python MySQL-Tutorial](https://pynative.com/python-mysql-tutorial/)
geht auf viele Details genauer ein.